# M5 Forecasting - Accuracy TimeSeriesSplit CV- Poisson Loss

Xiao Song

<https://xsong.ltd/en>     
[Kaggle profile](https://www.kaggle.com/rikdifos/)

The competition webpage: <https://www.kaggle.com/c/m5-forecasting-accuracy>

This notebook is mainly based on: [Simple LGBM GroupKFold CV](https://www.kaggle.com/ragnar123/simple-lgbm-groupkfold-cv/), shout out to the author [ragnar](https://www.kaggle.com/ragnar123/).



`sales_train_validation` contains d_1-d_1913 (2011-01-29 ~ 2011-04-24) sales data of 30490 rows.   
`sales_train_evaluation`  contains d_1-d_1941 (2011-01-29 ~ 2011-05-22) sales data of 30490 rows.      
`submission` has 60980 rows, public LB calulates `sales_train_validation`(d_1914-d_1941)  
Private LB evaluations `sales_train_evaluation`(d_1942-d_1968).

This notebook will output a 'submission1.csv' file.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import lightgbm as lgb
from datetime import datetime, timedelta
from sklearn import preprocessing, metrics

from sklearn.model_selection import GroupKFold, TimeSeriesSplit

import gc
import os
import time

In [2]:
def reduce_mem_usage(df, verbose=True):
    '''reduce RAM usage
    '''
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def read_data():
    '''data input
    '''
    print('Reading files...')
    calendar = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    sell_prices = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    sales_train_evaluation = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
    print('Sales train evaluation has {} rows and {} columns'.format(sales_train_evaluation.shape[0], sales_train_evaluation.shape[1]))
    submission = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sample_submission.csv')
    return calendar, sell_prices, sales_train_evaluation, submission


def melt_and_merge(calendar, sell_prices, sales_train_evaluation, submission, nrows = 55000000, merge = False):
    
    # melt sales data, get it ready for training
    sales_train_evaluation = pd.melt(sales_train_evaluation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    print('Melted sales train validation has {} rows and {} columns'.format(sales_train_evaluation.shape[0], sales_train_evaluation.shape[1]))
    sales_train_evaluation = reduce_mem_usage(sales_train_evaluation)
    
    # seperate test dataframes
    test1_rows = [row for row in submission['id'] if 'validation' in row]
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test1 = submission[submission['id'].isin(test1_rows)]
    test2 = submission[submission['id'].isin(test2_rows)]
    
    # change column names
    test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                      'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
    test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                      'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']
    
    # get product table
    product = sales_train_evaluation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    # merge with product table
    product['id'] = product['id'].str.replace('_evaluation','_validation')
    test1 = test1.merge(product, how = 'left', on = 'id') # validation
    product['id'] = product['id'].str.replace('_validation','_evaluation')
    test2 = test2.merge(product, how = 'left', on = 'id') # evaluation
    
    # 
    test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    
    sales_train_evaluation['part'] = 'train'
    test1['part'] = 'validation'
    test2['part'] = 'evaluation'
    
    data = pd.concat([sales_train_evaluation, test1, test2], axis = 0)
    
    del sales_train_evaluation, test1, test2
    
    # get only a sample for fst training
    data = data.loc[nrows:]
    
    # drop some calendar features
    # calendar.drop(['weekday', 'wday', 'month', 'year','snap_CA','snap_TX','snap_WI'], inplace = True, axis = 1)
    
    # delete test2 for now
    data = data[data['part'] != 'validation']
    
    if merge:
        # notebook crash with the entire dataset (maybee use tensorflow, dask, pyspark xD)
        data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
        data.drop(['weekday', 'wday', 'month', 'year','snap_CA','snap_TX','snap_WI'], inplace = True, axis = 1)
        # get the sell price data (this feature should be very important)
        data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
        print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    else: 
        pass
    
    data.to_pickle('data_clean.pkl')
    gc.collect()
    
    return data

In [3]:
cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']

def transform(data):
    '''data transformation
    '''
    start = time.time()
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        encoder = preprocessing.LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
    print('Data transformation costs %7.2f seconds'%(time.time()-start))
    return data

def simple_fe(data):
    '''do some feature engineering
    '''
    start = time.time()
    # rolling demand features
    data_fe = data[['id', 'demand']]
    
    window = 28
    periods = [7, 15, 30, 90]
    group = data_fe.groupby('id')['demand']
    # most recent lag data
    for period in periods:
        data_fe['demand_rolling_mean_t' + str(period)] = group.transform(lambda x: x.shift(window).rolling(period).mean())

    periods = [7, 90]
    for period in periods:
        data_fe['demand_rolling_std_t' + str(period)] = group.transform(lambda x: x.shift(window).rolling(period).std())
        
    # reduce memory
    data_fe = reduce_mem_usage(data_fe)
    # get time features
    data['date'] = pd.to_datetime(data['date'])
    time_features = ['year', 'month', 'week', 'day', 'dayofweek', 'dayofyear']
    dtype = np.int16
    for time_feature in time_features:
        data[time_feature] = getattr(data['date'].dt, time_feature).astype(dtype)
        
    # concat lag and rolling features with main table
    lag_rolling_features = [col for col in data_fe.columns if col not in ['id', 'demand']]
    data = pd.concat([data, data_fe[lag_rolling_features]], axis = 1)
    data['weekends'] = np.where((data['date'].dt.dayofweek) < 5, 0, 1)

    del data_fe
    gc.collect()
    print('Simple feature engineering costs %7.2f seconds'%(time.time()-start))
    return data

def run_lgb(data):
    '''cross validation using timeseries split
    '''
    start = time.time()
    data = data.sort_values('date')
    x_train = data[data['date'] <= '2016-05-22']
    y_train = x_train['demand']
    test = data[(data['date'] > '2016-05-22')]
    

    del data
    gc.collect()

    params = {
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'objective': 'poisson', # loss function
        'seed': 225,
        'learning_rate': 0.02,
        'lambda': 0.4, # l2 regularization
        'reg_alpha': 0.4, # l1 regularization
        'max_depth': 5, # max depth of decision trees
        'num_leaves': 64, #  number of leaves
        'bagging_fraction': 0.7, # bootstrap sampling
        'bagging_freq' : 1,
        'colsample_bytree': 0.7 # feature sampling
    }
    
      
    oof = np.zeros(len(x_train))
    preds = np.zeros(len(test))
    
    n_fold = 3 #3 for timely purpose of the kernel
    folds = TimeSeriesSplit(n_splits=n_fold) # use TimeSeriesSplit cv
    splits = folds.split(x_train, y_train)
    #feature_importances = pd.DataFrame()
    #feature_importances['feature'] = features
    for fold, (trn_idx, val_idx) in enumerate(splits):
        print(f'Training fold {fold + 1}')
        
        train_set = lgb.Dataset(x_train.iloc[trn_idx][features], y_train.iloc[trn_idx], categorical_feature = cat)
        val_set = lgb.Dataset(x_train.iloc[val_idx][features], y_train.iloc[val_idx], categorical_feature = cat)

        model = lgb.train(params, train_set, num_boost_round = 2400, early_stopping_rounds = 50, 
                          valid_sets = [val_set], verbose_eval = 50)
        #lgb.plot_importance(model, importance_type = 'gain', precision = 0,
                               # height = 0.5, figsize = (6, 10), title = '') 
        #feature_importances[f'fold_{fold + 1}'] = model.feature_importance()
        
        oof[val_idx] = model.predict(x_train.iloc[val_idx][features]) # cv prediction
        preds += model.predict(test[features]) / 3 # calculate mean prediction value of 3 models
        print('-' * 50)
        print('\n')
    #model.save_model('model.lgb') # save model
    del x_train
        
    print('3 folds cross-validation costs %7.2f seconds'%(time.time() - start))

    oof_rmse = np.sqrt(metrics.mean_squared_error(y_train, oof))
    print(f'Our out of folds rmse is {oof_rmse}')
    del y_train
        
    test = test[['id', 'date', 'demand']]
    test['demand'] = preds
    gc.collect()
    return test

def predict(test, submission):
    '''predict test and validation data label
    '''
    start = time.time()
    predictions = test[['id', 'date', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
    #predictions.to_csv('predictions.csv', index = False)

    prediction_val = predictions.copy()
    prediction_val['id'] = prediction_val['id'].str.replace('_evaluation','_validation') # change id to validation
    #prediction_val.to_csv('prediction_val.csv', index = False)
    
    concated = pd.concat([predictions, prediction_val])
    del predictions, prediction_val
    print('final dataset to train has {} rows and {} columns'.format(concated.shape[0], concated.shape[1]))
    concated.to_csv('submission1.csv', index = False)
    print('Data prediction costs %7.2f seconds'%(time.time() - start))

# define list of features
features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'sell_price', 'year', 
                'month', 'week', 'day', 'dayofweek', 'dayofyear', 'demand_rolling_mean_t7', 'demand_rolling_mean_t15', 'demand_rolling_mean_t30', 'demand_rolling_mean_t90',
                'demand_rolling_std_t7', 'demand_rolling_std_t90','weekends']

In [4]:
import seaborn as sns
def train_and_evaluate(): 
    '''run the all program
    '''
    calendar, sell_prices, sales_train_evaluation, submission = read_data()
    data = melt_and_merge(calendar, sell_prices, sales_train_evaluation, submission, nrows = 27500000, merge = True)
    data = transform(data)
    data['date'] = pd.to_datetime(data['date'])
    days = abs((data['date'].min() - data['date'].max()).days)
    # how many training data do we need to train with at least 2 years and consider lags
    need = 365 + 365 + 90 + 28
    print(f'We have {(days - 28)} days of training history')
    print(f'we have {(days - 28 - need)} days left')
    if (days - 28 - need) > 0:
        print('We have enought training data, lets continue')
    else:
        print('Get more training data, training can fail')
        
    data = simple_fe(data)
    data = reduce_mem_usage(data)

    print('Removing first 118 days')
    # eliminate the first 118 days of our train data because of lags
    min_date = data['date'].min() + timedelta(days = 118)
    data = data[data['date'] > min_date]
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']

    test = run_lgb(data)
    del data
    predict(test, submission) 
    #feature_importances['average'] = feature_importances[[f'fold_{fold_n + 1}' for fold_n in range(3)]].mean(axis=1)
    #feature_importances.to_csv('feature_importances.csv')

    #plt.figure(figsize=(16, 12))
    #sns.barplot(data=feature_importances.sort_values(by='average', ascending = False).head(20), x='average', y='feature')

In [ ]:
%%time
train_and_evaluate() 

```
Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train evaluation has 30490 rows and 1947 columns
Melted sales train validation has 59181090 rows and 8 columns
Mem. usage decreased to 3273.49 Mb (9.4% reduction)
Our final dataset to train has 32534810 rows and 17 columns
Data transformation costs  138.60 seconds
We have 1039 days of training history
we have 191 days left
We have enought training data, lets continue
Mem. usage decreased to 930.83 Mb (58.3% reduction)
Simple feature engineering costs  317.00 seconds
Mem. usage decreased to 2389.13 Mb (50.6% reduction)
Removing first 118 days
Training fold 1
Training until validation scores don't improve for 50 rounds
[50]	valid_0's rmse: 2.92764
[100]	valid_0's rmse: 2.61871
[150]	valid_0's rmse: 2.48988
[200]	valid_0's rmse: 2.43741
[250]	valid_0's rmse: 2.41602
[300]	valid_0's rmse: 2.40695
[350]	valid_0's rmse: 2.40337
[400]	valid_0's rmse: 2.40114
[450]	valid_0's rmse: 2.4004
[500]	valid_0's rmse: 2.39984
[550]	valid_0's rmse: 2.39913
[600]	valid_0's rmse: 2.39862
[650]	valid_0's rmse: 2.39786
[700]	valid_0's rmse: 2.39755
[750]	valid_0's rmse: 2.39721
[800]	valid_0's rmse: 2.39712
[850]	valid_0's rmse: 2.39687
[900]	valid_0's rmse: 2.39651
Early stopping, best iteration is:
[889]	valid_0's rmse: 2.3963
--------------------------------------------------


Training fold 2
Training until validation scores don't improve for 50 rounds
[50]	valid_0's rmse: 2.90773
[100]	valid_0's rmse: 2.59335
[150]	valid_0's rmse: 2.45543
[200]	valid_0's rmse: 2.39586
[250]	valid_0's rmse: 2.36832
[300]	valid_0's rmse: 2.3551
[350]	valid_0's rmse: 2.34866
[400]	valid_0's rmse: 2.34514
[450]	valid_0's rmse: 2.3428
[500]	valid_0's rmse: 2.34088
[550]	valid_0's rmse: 2.33887
[600]	valid_0's rmse: 2.33775
[650]	valid_0's rmse: 2.3368
[700]	valid_0's rmse: 2.33575
[750]	valid_0's rmse: 2.33509
[800]	valid_0's rmse: 2.33459
[850]	valid_0's rmse: 2.33367
[900]	valid_0's rmse: 2.33292
[950]	valid_0's rmse: 2.33142
[1000]	valid_0's rmse: 2.33048
[1050]	valid_0's rmse: 2.32928
[1100]	valid_0's rmse: 2.32855
[1150]	valid_0's rmse: 2.32763
[1200]	valid_0's rmse: 2.32685
[1250]	valid_0's rmse: 2.32544
[1300]	valid_0's rmse: 2.32483
[1350]	valid_0's rmse: 2.3235
[1400]	valid_0's rmse: 2.32323
[1450]	valid_0's rmse: 2.32276
[1500]	valid_0's rmse: 2.32192
[1550]	valid_0's rmse: 2.32112
[1600]	valid_0's rmse: 2.32035
[1650]	valid_0's rmse: 2.31982
[1700]	valid_0's rmse: 2.31956
[1750]	valid_0's rmse: 2.31903
[1800]	valid_0's rmse: 2.31878
[1850]	valid_0's rmse: 2.31813
[1900]	valid_0's rmse: 2.31716
[1950]	valid_0's rmse: 2.31617
[2000]	valid_0's rmse: 2.31548
[2050]	valid_0's rmse: 2.31468
[2100]	valid_0's rmse: 2.31359
[2150]	valid_0's rmse: 2.31278
[2200]	valid_0's rmse: 2.31204
[2250]	valid_0's rmse: 2.31149
[2300]	valid_0's rmse: 2.3107
[2350]	valid_0's rmse: 2.31028
[2400]	valid_0's rmse: 2.30987
Did not meet early stopping. Best iteration is:
[2400]	valid_0's rmse: 2.30987
--------------------------------------------------


Training fold 3
Training until validation scores don't improve for 50 rounds
[50]	valid_0's rmse: 2.84103
[100]	valid_0's rmse: 2.52998
[150]	valid_0's rmse: 2.40025
[200]	valid_0's rmse: 2.34677
[250]	valid_0's rmse: 2.32351
[300]	valid_0's rmse: 2.31261
[350]	valid_0's rmse: 2.30719
[400]	valid_0's rmse: 2.30452
[450]	valid_0's rmse: 2.30257
[500]	valid_0's rmse: 2.29983
[550]	valid_0's rmse: 2.29799
[600]	valid_0's rmse: 2.29671
[650]	valid_0's rmse: 2.29576
[700]	valid_0's rmse: 2.29507
[750]	valid_0's rmse: 2.29452
[800]	valid_0's rmse: 2.29427
[850]	valid_0's rmse: 2.29374
[900]	valid_0's rmse: 2.29309
[950]	valid_0's rmse: 2.29264
[1000]	valid_0's rmse: 2.29217
[1050]	valid_0's rmse: 2.29189
[1100]	valid_0's rmse: 2.29147
[1150]	valid_0's rmse: 2.29113
[1200]	valid_0's rmse: 2.29055
[1250]	valid_0's rmse: 2.29002
[1300]	valid_0's rmse: 2.28979
[1350]	valid_0's rmse: 2.28946
[1400]	valid_0's rmse: 2.28906
[1450]	valid_0's rmse: 2.28871
[1500]	valid_0's rmse: 2.28821
[1550]	valid_0's rmse: 2.28782
[1600]	valid_0's rmse: 2.2876
[1650]	valid_0's rmse: 2.28725
[1700]	valid_0's rmse: 2.28699
[1750]	valid_0's rmse: 2.28664
[1800]	valid_0's rmse: 2.28608
[1850]	valid_0's rmse: 2.28563
[1900]	valid_0's rmse: 2.28535
[1950]	valid_0's rmse: 2.28495
[2000]	valid_0's rmse: 2.28451
[2050]	valid_0's rmse: 2.28397
[2100]	valid_0's rmse: 2.28357
[2150]	valid_0's rmse: 2.28319
[2200]	valid_0's rmse: 2.2829
[2250]	valid_0's rmse: 2.28259
[2300]	valid_0's rmse: 2.28238
[2350]	valid_0's rmse: 2.28224
[2400]	valid_0's rmse: 2.28193
Did not meet early stopping. Best iteration is:
[2400]	valid_0's rmse: 2.28193
--------------------------------------------------


3 folds cross-validation costs 11992.92 seconds
Our out of folds rmse is 2.823272209788697
final dataset to train has 60980 rows and 29 columns
Data prediction costs    9.68 seconds
CPU times: user 13h 5min 33s, sys: 4min 16s, total: 13h 9min 49s
Wall time: 3h 33min 1s
```